In [1]:
import os
import pandas as pd
import requests
from PIL import Image
from io import BytesIO
from sklearn.cluster import KMeans
import numpy as np
import cv2
import matplotlib.pyplot as plt

# Load your cleaned dataframe
df = pd.read_csv("fitzpatrick17k.csv")
df = df.dropna(subset=['url']).reset_index(drop=True)

# Filter only Psoriasis images
psoriasis_df = df[df['label'].str.lower() == 'psoriasis']
os.makedirs("images/psoriasis", exist_ok=True)


In [2]:
unique_labels = df['label'].unique()
label_counts = df['label'].value_counts()
print(label_counts)


label
psoriasis                      653
squamous cell carcinoma        581
lichen planus                  491
basal cell carcinoma           468
allergic contact dermatitis    430
                              ... 
port wine stain                 59
erythema elevatum diutinum      55
xanthomas                       53
pustular psoriasis              53
pilomatricoma                   53
Name: count, Length: 114, dtype: int64


In [27]:
# # Downloading all of the Psoriasis images from the url field

# def download_image(url, save_path):
#     headers = {'User-Agent': 'Mozilla/5.0'}
#     try:
#         response = requests.get(url, headers=headers, timeout=10)
#         if response.status_code == 200:
#             try:
#                 image = Image.open(BytesIO(response.content)).convert("RGB")
#                 image.save(save_path)
#                 return True
#             except Exception as e:
#                 print(f"Invalid image at {url} | {e}")
#         else:
#             print(f"Bad status ({response.status_code}) for {url}")
#     except Exception as e:
#         print(f"Failed to download {url} | {e}")
#     return False


# # Download all Psoriasis images
# for idx, row in psoriasis_df.iterrows():
#     img_name = f"images/psoriasis/psoriasis_{idx}.jpg"
#     if not os.path.exists(img_name):
#         download_image(row['url'], img_name)


In [4]:
import os

ds_store_path = "images/psoriasis/.DS_Store"
if os.path.exists(ds_store_path):
    os.remove(ds_store_path)
    print(".DS_Store file deleted.")
else:
    print(".DS_Store not found.")


.DS_Store file deleted.


In [5]:
folder_path = "images/psoriasis"
all_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]
print(f"Total files in '{folder_path}': {len(all_files)}")


Total files in 'images/psoriasis': 653


In [ ]:
#############################################################################################################################################

In [10]:
import os
import cv2
import torch
import numpy as np
from PIL import Image
from torchvision import transforms

# Add U-2-Net code folder to path
import sys
sys.path.append("U-2-Net")  # make sure this matches your actual folder name

# Now import the model
from model import U2NETP

# Load model
def load_model(model_path='U-2-Net/saved_models/u2net/u2netp.pth'):
    from model import U2NETP
    net = U2NETP(3, 1)  # Use the correct small model
    net.load_state_dict(torch.load(model_path, map_location='cpu'))
    net.eval()
    return net

# Preprocess image
def preprocess_image(img_path):
    image = Image.open(img_path).convert("RGB").resize((320, 320))
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])])
    return transform(image).unsqueeze(0), image

# Convert model output to usable mask
def postprocess_mask(pred):
    pred = pred.squeeze().cpu().data.numpy()
    pred = (pred - pred.min()) / (pred.max() - pred.min())
    return (pred * 255).astype(np.uint8)

# Apply mask to image
def apply_mask(image, mask):
    image = image.resize(mask.size)
    image_np = np.array(image)
    mask_np = np.array(mask)
    mask_3c = np.stack([mask_np]*3, axis=-1)
    result = np.where(mask_3c > 128, image_np, 0)
    return Image.fromarray(result)

# Batch process folder
def remove_background_from_folder(input_folder, output_folder, model):
    os.makedirs(output_folder, exist_ok=True)
    files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    for file in files:
        img_path = os.path.join(input_folder, file)
        img_tensor, original_img = preprocess_image(img_path)

        with torch.no_grad():
            d1, *_ = model(img_tensor)
            mask = postprocess_mask(d1)

        mask_img = Image.fromarray(mask).resize(original_img.size)
        final_img = apply_mask(original_img, mask_img)
        final_img.save(os.path.join(output_folder, file))

# Run it
model = load_model()
remove_background_from_folder("images/psoriasis", "images/psoriasis_nobg", model)


/Users/anuragpathak/Downloads/PemphigusImageProject/U-2-Net/model/u2net.py:23: UserWarning: `nn.functional.upsample` is deprecated. Use `nn.functional.interpolate` instead.
  src = F.upsample(src,size=tar.shape[2:],mode='bilinear')


In [ ]:
##########################################################